In [15]:
# import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import row_number, lit
from pyspark.sql.window import Window
import pickle

In [3]:
spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/18 19:19:01 WARN Utils: Your hostname, LAPTOP-0C387IAH resolves to a loopback address: 127.0.1.1; using 172.21.209.100 instead (on interface eth0)
23/09/18 19:19:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/18 19:19:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = StructType([
    StructField("Name", StringType()),
    StructField("Age", IntegerType()),
    StructField("Gender", StringType()),
    StructField("Occupation", StringType()),
    StructField("Hobby", StringType()),
])


In [7]:
df = spark.read.format('csv').schema(schema).load('3_data.csv')

In [8]:
df.show()

+-------+----+------+----------+--------------------+
|   Name| Age|Gender|Occupation|               Hobby|
+-------+----+------+----------+--------------------+
|Michael|null|  Male|     Sales|“Loves to play gu...|
| Robert|  35|  Male|     Sales|“Enjoys hiking an...|
|   Jane|  28|Female| Marketing|“Enjoys hiking an...|
|    Bob|  42|  Male|   Finance|     “Likes to read\|
+-------+----+------+----------+--------------------+



In [11]:
df.write.option("header",True).csv('3_answer.csv')

In [16]:
w = Window().orderBy(lit('A'))
df = df.withColumn("row_num", row_number().over(w))

In [17]:
df.show()

23/09/18 19:32:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:32:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:32:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:32:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:32:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----+------+----------+--------------------+-------+
|   Name| Age|Gender|Occupation|               Hobby|row_num|
+-------+----+------+----------+--------------------+-------+
|Michael|null|  Male|     Sales|“Loves to play gu...|      1|
| Robert|  35|  Male|     Sales|“Enjoys hiking an...|      2|
|   Jane|  28|Female| Marketing|“Enjoys hiking an...|      3|
|    Bob|  42|  Male|   Finance|     “Likes to read\|      4|
+-------+----+------+----------+--------------------+-------+



In [27]:
null_rows = (df
             .select('row_num')
             .filter(df.Name.isNull() | df.Age.isNull() | df.Gender.isNull()| df.Occupation.isNull())
             .collect()
)

23/09/18 19:39:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:39:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:39:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:39:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/18 19:39:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [29]:
for row in null_rows:
    print(row.row_num)


1
